<a href="https://colab.research.google.com/github/shizafatima/Quarter02_Projects/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.1 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_key = userdata.get("PINECONE_KEY")

pc = Pinecone(api_key=pinecone_key)

In [ ]:
index_name = "project02-rag"  # change if desired

if index_name not in pc.list_indexes():
  pc.create_index(
      name=index_name,
      dimension=768,
      metric="cosine",
      spec=ServerlessSpec(cloud="aws", region="us-east-1"),
  )
else:
    print(f"Index '{index_name}' exists.")

index = pc.Index(index_name)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [ ]:
!pip install -qU langchain-community

In [ ]:
%pip install -qU pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load documents
loader = PyPDFLoader("Anatomy of organs.pdf")  # Replace with your file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'Anatomy of organs.pdf', 'page': 0}, page_content='1-SHORTNOTEONLIVER:-\nTheliverliesintherightupperquadrantoftheabdomen,immediatelyinferiortothediaphragm.Theliveriscoveredbyaconnectivetissuecapsuleandlayerofvisceralperitoneum.'), Document(metadata={'source': 'Anatomy of organs.pdf', 'page': 0}, page_content='Theliveriscomposedoffourincompletelyseparatedlobes.Themajorlobesaretherightlobeandtheleftlobe.Itispresentontheposteriorside.Therightlobeissubdividedintotwo:\n1-Caudatelobealongwithligamentumvenosum.\n2-Quadratelobealongwithligamentumteres.'), Document(metadata={'source': 'Anatomy of organs.pdf', 'page': 0}, page_content='2-Quadratelobealongwithligamentumteres.\nIthasoneborderi.einferiorborderandthreesurfacesi.eAnteriorsurface,rightsurface,superiorsurface.\nOntheposteriorsurfacetwoligamentsarepresent.\n1-Ligamentumvenosum.\n2-Ligamentumteres.'), Document(metadata={'source': 'Anatomy of organs.pdf', 'page': 0}, page_content='1-Ligamentumvenosum.\n2-Liga

In [ ]:
len(documents)

10

In [ ]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['f81e86c7-2c4d-4891-bba3-b9f7780b8853',
 '31e6dc6a-c743-4ef2-b928-83abc72cb0df',
 'df5c6848-64ca-44b8-9e6a-9ccfe31dd3e3',
 'e6aa3ac0-db9c-4605-8856-bcd91f74c7bc',
 '31017630-3ddf-442d-9b62-4149503ba382',
 '654483b3-1475-403f-82e3-2b30cc6a457c',
 '70dbef6a-ad41-43a8-8122-f75c4e8b8242',
 'a7d58d47-3e31-4239-8ddc-46f86bb36bff',
 '5f4e9927-bbe8-4fc8-918e-953a7284b63a',
 '7c81f431-6476-434a-b83d-2e90c0282c05']

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    # Generate a unique ID for each document (e.g., UUID or hash)
    doc_id = str(hash(doc.page_content))
    # Upsert the vector with a unique ID
    index.upsert(vectors=[(doc_id, vector, {"text": doc.page_content})])

100%|██████████| 72/72 [00:16<00:00,  4.30it/s]


In [ ]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


# Convert it into a retriever compatible with RetrievalQA
retriever = vector_store.as_retriever()

<ipython-input-23-b373e6fbbf23>:4: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

llm : ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key= GOOGLE_API_KEY,
    temprature = 0.9,
    max_tokens = 200
)


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="refine",  # Other options: "map_reduce", "refine"
    retriever=retriever
)

In [ ]:
query = "How many parts of Pancreas? Name them."
response = qa_chain.run(query)
print(response)

The new context provides information about the blood supply to the pancreas, which is interesting but **not relevant** to the question of how many parts the pancreas has and what they are called.

Therefore, the original answer remains the best and most accurate response to the question:

**The pancreas has 4 parts. They are:**

1.  **Head**
2.  **Neck**
3.  **Body**
4.  **Tail**

